### Climate downscaling using CDDLT package

In [1]:
# !pip install --index-url https://test.pypi.org/simple/ \
#              --extra-index-url https://pypi.org/simple \
#              cddlt --quiet

In [2]:
import os
import argparse
import torch
import torchmetrics
import cddlt

from cddlt.datasets.rekis_dataset import ReKIS
from cddlt.datasets.cordex_dataset import CORDEX
from cddlt.dataloaders.downscaling_transform import DownscalingTransform

from cddlt.models.bicubic import Bicubic
from cddlt.models.espcn import ESPCN
from cddlt.models.edsr import EDSR
from cddlt.models.srcnn import SRCNN
from cddlt.models.fno import FNO
from cddlt.models.swinir import SwinIR

/home/kostape4/downscale/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kostape4/downscale/venv/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=8, type=int)
parser.add_argument("--epochs", default=100, type=int)
parser.add_argument("--seed", default=42, type=int)
parser.add_argument("--threads", default=4, type=int)
parser.add_argument("--upscale_factor", default=10, type=int)
parser.add_argument("--lr", default=0.001, type=float)
parser.add_argument("--logdir", default="logs", type=str)
parser.add_argument("--variables", default=["TM"], type=list)

_StoreAction(option_strings=['--variables'], dest='variables', nargs=None, const=None, default=['TM'], type=<class 'list'>, choices=None, required=False, help=None, metavar=None)

In [4]:
REKIS_DATA_PATH = "/mnt/personal/kostape4/data/climate/rekis/"
CORDEX_DATA_PATH = "/mnt/personal/kostape4/data/climate//cordex/"

In [5]:
args = parser.parse_args([])
cddlt.startup(args)

### ReKIS

In [6]:
rekis = ReKIS(
    data_path=REKIS_DATA_PATH,
    variables=args.variables,
    train_len=("1979-01-01", "2000-01-01"),
    dev_len=("2000-01-01", "2005-01-01"),
    test_len=("2005-01-01", "2012-01-01"), ## value framework input
    resampling="cubic_spline"
)

Loading 63 NetCDF file(s)...
Loaded data shape: {'time': 7670, 'northing': 400, 'easting': 400}
Time range: 1979-01-01 00:00:00 to 1999-12-31 00:00:00
Loading 63 NetCDF file(s)...
Loaded data shape: {'time': 1827, 'northing': 400, 'easting': 400}
Time range: 2000-01-01 00:00:00 to 2004-12-31 00:00:00
Loading 63 NetCDF file(s)...
Loaded data shape: {'time': 2556, 'northing': 400, 'easting': 400}
Time range: 2005-01-01 00:00:00 to 2011-12-31 00:00:00

ReKIS dataset initalized.
train size: (7670)
dev size: (1827)
test size: (2556)



In [7]:
rekis_train = DownscalingTransform(dataset=rekis.train).dataloader(args.batch_size, shuffle=True)
rekis_dev = DownscalingTransform(dataset=rekis.dev).dataloader(args.batch_size)
rekis_test = DownscalingTransform(dataset=rekis.test).dataloader(args.batch_size)

### Models

#### Bicubic RMSE baseline

In [14]:
bicubic = Bicubic(
    upscale_factor=args.upscale_factor
)

In [16]:
bicubic.configure(
    optimizer = None,
    scheduler = None,
    loss = torchmetrics.MeanSquaredError(squared=False),
    args = args
)

Bicubic(
  (loss): MeanSquaredError()
)

In [17]:
bicubic.evaluate(rekis_dev, print_loss=True, epochs=args.epochs)

Evaluation - dev_loss: 0.2569


#### Bicubic L1 baseline

In [8]:
bicubic = Bicubic(
    upscale_factor=args.upscale_factor
)

In [9]:
bicubic.configure(
    optimizer = None,
    scheduler = None,
    loss = torch.nn.L1Loss(),
    args = args
)

Bicubic(
  (loss): L1Loss()
)

In [10]:
bicubic.evaluate(rekis_dev, print_loss=True, epochs=args.epochs)

Evaluation - dev_loss: 0.1518


#### SRCNN

In [18]:
srcnn = SRCNN(
    n_channels=1,
    upscale_factor=args.upscale_factor
)

In [19]:
srcnn.configure(
    optimizer = torch.optim.AdamW(params=srcnn.parameters(), lr=args.lr),
    scheduler = None,
    loss = torchmetrics.MeanSquaredError(squared=False),
    args = args
)

SRCNN(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (1): ReLU()
    (2): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
    (4): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (loss): MeanSquaredError()
)

In [20]:
srcnn.fit(rekis_train, rekis_dev, args.epochs)

1/100:   0%|                                                                                                                                                                            | 0/120 [00:00<?, ?it/s]/home/kostape4/downscale/venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/kostape4/downscale/venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac

Epoch 1/100 - train_loss: 0.8270, dev_loss: 0.3375 (47.87s)


Epoch 2/100 - train_loss: 0.4741, dev_loss: 0.3103 (47.20s)


Epoch 3/100 - train_loss: 0.4667, dev_loss: 0.3316 (47.13s)


Epoch 4/100 - train_loss: 0.3985, dev_loss: 0.5010 (47.13s)


...

Epoch 96/100 - train_loss: 0.2679, dev_loss: 0.2356 (47.11s)


Epoch 97/100 - train_loss: 0.2516, dev_loss: 0.2334 (47.12s)


Epoch 98/100 - train_loss: 0.2484, dev_loss: 0.2736 (47.13s)


Epoch 99/100 - train_loss: 0.2566, dev_loss: 0.2380 (47.13s)


Epoch 100/100 - train_loss: 0.2489, dev_loss: 0.2441 (47.11s)


#### ESPCN

In [ ]:
args = parser.parse_args([])
cddlt.startup(args)

In [21]:
espcn = ESPCN(
    n_channels=1,
    upscale_factor=args.upscale_factor,
)

In [22]:
espcn.configure(
    optimizer = torch.optim.AdamW(params=espcn.parameters(), lr=args.lr),
    scheduler = None,
    loss = torchmetrics.MeanSquaredError(squared=False),
    args = args
)

ESPCN(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Tanh()
    (4): Conv2d(32, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): PixelShuffle(upscale_factor=10)
  )
  (loss): MeanSquaredError()
)

In [23]:
espcn.fit(rekis_train, rekis_dev, args.epochs)

Epoch 1/100 - train_loss: 4.4526, dev_loss: 1.3290 (4.93s)


Epoch 2/100 - train_loss: 1.1119, dev_loss: 0.8996 (3.34s)


Epoch 3/100 - train_loss: 0.8910, dev_loss: 0.7705 (3.35s)


Epoch 4/100 - train_loss: 0.7864, dev_loss: 0.8812 (3.15s)


Epoch 5/100 - train_loss: 0.6399, dev_loss: 0.6064 (3.40s)


...

Epoch 96/100 - train_loss: 0.3456, dev_loss: 0.2957 (3.33s)


Epoch 97/100 - train_loss: 0.2956, dev_loss: 0.3610 (3.36s)


Epoch 98/100 - train_loss: 0.5555, dev_loss: 0.2966 (3.22s)


Epoch 99/100 - train_loss: 0.2868, dev_loss: 0.2881 (3.12s)


Epoch 100/100 - train_loss: 0.4074, dev_loss: 0.2717 (3.17s)


#### FNO

In [24]:
fno = FNO(
    n_channels=1,
    upscale_factor=args.upscale_factor
)

In [25]:
fno.configure(
    optimizer = torch.optim.AdamW(params=espcn.parameters(), lr=args.lr),
    scheduler = None,
    loss = torchmetrics.MeanSquaredError(squared=False),
    args = args
)

FNO(
  (P): Linear(in_features=1, out_features=32, bias=True)
  (spectral_convs): ModuleList(
    (0-3): 4 x SpectralConv2d()
  )
  (weights): ModuleList(
    (0-3): 4 x Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  )
  (Q): Linear(in_features=32, out_features=1, bias=True)
  (loss): MeanSquaredError()
)

In [26]:
fno.fit(rekis_train, rekis_dev, args.epochs)

Epoch 1/100 - train_loss: 11.5460, dev_loss: 10.7557 (134.69s)


Epoch 2/100 - train_loss: 11.5400, dev_loss: 10.7557 (133.12s)


Epoch 3/100 - train_loss: 11.5461, dev_loss: 10.7557 (133.11s)


Epoch 4/100 - train_loss: 11.5436, dev_loss: 10.7557 (133.12s)


Epoch 5/100 - train_loss: 11.5472, dev_loss: 10.7557 (133.12s)


Epoch 6/100 - train_loss: 11.5418, dev_loss: 10.7557 (133.12s)


Epoch 7/100 - train_loss: 11.5486, dev_loss: 10.7557 (133.12s)


KeyboardInterrupt: 

#### EDSR

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=32, type=int)
parser.add_argument("--epochs", default=60, type=int)
parser.add_argument("--seed", default=42, type=int)
parser.add_argument("--threads", default=0, type=int)
parser.add_argument("--upscale_factor", default=10, type=int)
parser.add_argument("--lr", default=1e-4, type=float)
parser.add_argument("--wd", default=1e-5, type=float)
parser.add_argument("--logdir", default="logs", type=str)
parser.add_argument("--variables", default=["TM"], type=list)

_StoreAction(option_strings=['--variables'], dest='variables', nargs=None, const=None, default=['TM'], type=<class 'list'>, choices=None, required=False, help=None, metavar=None)

In [12]:
args = parser.parse_args([])
cddlt.startup(args)

In [13]:
edsr = EDSR(
    channels=1,
    scale_factor=10
)

In [15]:
optimizer = torch.optim.AdamW(params=edsr.parameters(), lr=args.lr, weight_decay=args.wd)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs)

In [16]:
edsr.configure(
    optimizer = optimizer,
    scheduler = scheduler,
    loss = torch.nn.L1Loss(),
    args = args
)

EDSR(
  (loss): L1Loss()
  (head): Sequential(
    (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (body): Sequential(
    (0): ResidualBlock(
      (residual_block): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): ResidualBlock(
      (residual_block): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): ResidualBlock(
      (residual_block): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (3): ResidualBlock(
      (residual_block): Sequential(
        (0): Conv2d(256,

In [ ]:
edsr.fit(rekis_train, rekis_dev, args.epochs)

1/60:   0%|                                                                      | 0/240 [00:00<?, ?it/s]/home/kostape4/downscale/venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/kostape4/downscale/venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
                                                                                            

Epoch 1/60 - train_loss: 0.6536, dev_loss: 0.3353 (431.99s)


Epoch 2/60 - train_loss: 0.2577, dev_loss: 0.8569 (424.62s)


Epoch 3/60 - train_loss: 0.2534, dev_loss: 0.1974 (425.25s)


Epoch 4/60 - train_loss: 0.2158, dev_loss: 0.2896 (424.68s)


...

Epoch 29/60 - train_loss: 0.0880, dev_loss: 0.1204 (423.85s)


Epoch 30/60 - train_loss: 0.0857, dev_loss: 0.1078 (423.77s)


Epoch 31/60 - train_loss: 0.0750, dev_loss: 0.0554 (425.50s)


Epoch 32/60 - train_loss: 0.0815, dev_loss: 0.1888 (424.85s)


33/60:  20%|███████▍                             | 48/240 [01:17<05:18,  1.66s/it]

#### DeepESDtas

In [10]:
class DeepESDtas(cddlt.DLModule):

    """
    DeepESD model as proposed in Baño-Medina et al. 2024 for temperature
    downscasling. This implementation allows for a deterministic (MSE-based)
    and stochastic (NLL-based) definition.

    Baño-Medina, J., Manzanas, R., Cimadevilla, E., Fernández, J., González-Abad,
    J., Cofiño, A. S., and Gutiérrez, J. M.: Downscaling multi-model climate projection
    ensembles with deep learning (DeepESD): contribution to CORDEX EUR-44, Geosci. Model
    Dev., 15, 6747–6758, https://doi.org/10.5194/gmd-15-6747-2022, 2022.

    Parameters
    ----------
    x_shape : tuple
        Shape of the data used as predictor. This must have dimension 4
        (time, channels/variables, lon, lat).

    y_shape : tuple
        Shape of the data used as predictand. This must have dimension 2
        (time, gridpoint)

    filters_last_conv : int
        Number of filters/kernels of the last convolutional layer

    stochastic: bool
        If set to True, the model is composed of two final dense layers computing
        the mean and log fo the variance. Otherwise, the models is composed of one
        final layer computing the values.
    """


    def __init__(self, x_shape: tuple, y_shape: tuple,
                 filters_last_conv: int, stochastic: bool):

        super(DeepESDtas, self).__init__()

        if (len(x_shape) != 4) or (len(y_shape) != 2):
            error_msg =\
            'X and Y data must have a dimension of length 4'
            'and 2, correspondingly'

            raise ValueError(error_msg)

        self.x_shape = x_shape
        self.y_shape = y_shape
        self.filters_last_conv = filters_last_conv
        self.stochastic = stochastic

        self.conv_1 = torch.nn.Conv2d(in_channels=self.x_shape[1],
                                      out_channels=50,
                                      kernel_size=3,
                                      padding=1)

        self.conv_2 = torch.nn.Conv2d(in_channels=50,
                                      out_channels=25,
                                      kernel_size=3,
                                      padding=1)

        self.conv_3 = torch.nn.Conv2d(in_channels=25,
                                      out_channels=self.filters_last_conv,
                                      kernel_size=3,
                                      padding=1)

        if self.stochastic:
            self.out_mean = torch.nn.Linear(in_features=\
                                            self.x_shape[2] * self.x_shape[3] * self.filters_last_conv,
                                            out_features=self.y_shape[1])

            self.out_log_var = torch.nn.Linear(in_features=\
                                               self.x_shape[2] * self.x_shape[3] * self.filters_last_conv,
                                               out_features=self.y_shape[1])

        else:
            self.out = torch.nn.Linear(in_features=\
                                       self.x_shape[2] * self.x_shape[3] * self.filters_last_conv,
                                       out_features=self.y_shape[1])

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        x = self.conv_1(x)
        x = torch.relu(x)

        x = self.conv_2(x)
        x = torch.relu(x)

        x = self.conv_3(x)
        x = torch.relu(x)

        x = torch.flatten(x, start_dim=1)

        if self.stochastic:
            mean = self.out_mean(x)
            log_var = self.out_log_var(x)
            out = torch.cat((mean, log_var), dim=1)
        else:
            out = self.out(x)
        
        return out

In [11]:
from typing import Dict, Tuple
from cddlt.models.deepesdtas import DeepESDtas

In [12]:
import torch
from typing import Type, Dict, Union, Any, List, Tuple, Optional, Callable

class DownscalingTransform:
    def __init__(
        self, 
        dataset: Type[torch.utils.data.Dataset],
        transform: Optional[Callable] = None,
        collate_fn: Optional[Callable] = None,
    ) -> None:
        self.dataset = dataset
        self.transform = transform if transform is not None else self.default_transform
        self.collate_fn = collate_fn if collate_fn is not None else self.default_collate_fn

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, index: int) -> Dict[str, Union[torch.Tensor, Any]]:
        sample = self.dataset[index]
        return self.transform(sample)
    
    @staticmethod
    def default_transform(sample: Dict[str, torch.Tensor]) -> Tuple[torch.Tensor, torch.Tensor]:
        return sample["input"], sample["target"]

    @staticmethod
    def default_collate_fn(batch: List[Tuple[torch.Tensor, torch.Tensor]]) -> Tuple[torch.Tensor, torch.Tensor]:
        inputs = torch.stack([item[0] for item in batch])
        targets = torch.stack([item[1] for item in batch])
        return inputs, targets

    def dataloader(self, batch_size: int, shuffle: bool = False, num_workers: int = 0) -> torch.utils.data.DataLoader:
        return torch.utils.data.DataLoader(
            self, 
            batch_size=batch_size, 
            shuffle=shuffle,
            collate_fn=self.collate_fn,
            num_workers=num_workers
        )

In [13]:
def sample_transform(sample: Dict[str, torch.Tensor]) -> Tuple[torch.Tensor, torch.Tensor]:
    inputs = sample[0]
    targets = sample[1]
    B, H, W = targets.shape
    targets = targets.view(B, H * W).squeeze(1)
    return inputs, targets

In [14]:
def collate_fn(batch: List[Tuple[torch.Tensor, torch.Tensor]]) -> Tuple[torch.Tensor, torch.Tensor]:
    inputs = torch.stack([item[0] for item in batch])
    targets = torch.stack([item[1] for item in batch]).squeeze(1)
    return inputs, targets.squeeze(1)

In [15]:
rekis_deepesd_train = DownscalingTransform(
    dataset=rekis.train, 
    transform=sample_transform,
    collate_fn=collate_fn
).dataloader(args.batch_size, shuffle=True)

rekis_deepesd_dev = DownscalingTransform(
    dataset=rekis.dev, 
    transform=sample_transform,
    collate_fn=collate_fn
).dataloader(args.batch_size)

In [16]:
deepesdtas = DeepESDtas(
    x_shape = (..., 1, 40, 40),
    y_shape = (..., 160000),
    filters_last_conv = 4,
    stochastic = False
)

In [17]:
deepesdtas.configure(
    optimizer = torch.optim.Adam(params=deepesdtas.parameters(), lr=args.lr),
    scheduler = None,
    loss = torchmetrics.MeanSquaredError(squared=False),
    args = args,
)

DeepESDtas(
  (conv_1): Conv2d(1, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(50, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3): Conv2d(25, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (out): Linear(in_features=6400, out_features=160000, bias=True)
  (loss): MeanSquaredError()
)

In [ ]:
deepesdtas.fit(rekis_deepesd_train, rekis_deepesd_dev, args.epochs)

Epoch 1/100 - train_loss: 0.9945, dev_loss: 0.5351 (126.20s)


Epoch 2/100 - train_loss: 0.5234, dev_loss: 0.3506 (125.22s)


Epoch 3/100 - train_loss: 0.3756, dev_loss: 0.5112 (114.57s)


Epoch 4/100 - train_loss: 0.3325, dev_loss: 0.3784 (114.49s)


Epoch 5/100 - train_loss: 0.2986, dev_loss: 0.2090 (125.07s)


...

Epoch 93/100 - train_loss: 0.0895, dev_loss: 0.0769 (114.51s)


Epoch 94/100 - train_loss: 0.0877, dev_loss: 0.0764 (114.52s)


Epoch 95/100 - train_loss: 0.0957, dev_loss: 0.0876 (114.52s)


Epoch 96/100 - train_loss: 0.0910, dev_loss: 0.0877 (114.51s)


97/100:  98%|█████████████████████████████████████████████████▉ | 938/959 [01:50<00:02,  8.51it/s]

In [ ]:
deepesdtas.load_weights(os.path.join(args.logdir, deepesdtas.model_name))
deepesdtas.evaluate(rekis_deepesd_dev, print_loss=True)

#### SwinIR

In [8]:
from typing import List

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=32, type=int)
parser.add_argument("--epochs", default=60, type=int)
parser.add_argument("--seed", default=42, type=int)
parser.add_argument("--threads", default=0, type=int)
parser.add_argument("--upscale_factor", default=10, type=int)
parser.add_argument("--embedding_dim", default=180, type=int)
parser.add_argument("--window_size", default=8, type=int)
parser.add_argument("--depths", default=[6, 6, 6, 6, 6, 6], type=List[int])
parser.add_argument("--num_heads", default=[6, 6, 6, 6, 6, 6], type=List[int])
parser.add_argument("--lr", default=1e-4, type=float)
parser.add_argument("--wd", default=1e-5, type=float)
parser.add_argument("--logdir", default="logs", type=str)
parser.add_argument("--variables", default=["TM"], type=list)

_StoreAction(option_strings=['--variables'], dest='variables', nargs=None, const=None, default=['TM'], type=<class 'list'>, choices=None, required=False, help=None, metavar=None)

In [11]:
args = parser.parse_args([])
cddlt.startup(args)

In [12]:
swin = SwinIR(
    img_size=40,
    in_chans=len(args.variables),
    window_size=args.window_size,
    embed_dim=args.embedding_dim,
    depths=args.depths,
    num_heads=args.num_heads,
    upsampler="pixelshuffle",
    upscale=args.upscale_factor
)

/home/kostape4/downscale/venv/lib/python3.12/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [13]:
optimizer = torch.optim.AdamW(params=swin.parameters(), lr=args.lr, weight_decay=args.wd)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs)

In [14]:
swin.configure(
    optimizer = optimizer,
    scheduler = scheduler,
    loss = torch.nn.L1Loss(),
    args = args
)

SwinIR(
  (conv_first): Conv2d(1, 180, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (patch_embed): PatchEmbed(
    (norm): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
  )
  (patch_unembed): PatchUnEmbed()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): RSTB(
      (residual_group): BasicLayer(
        dim=180, input_resolution=(40, 40), depth=6
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=180, input_resolution=(40, 40), num_heads=6, window_size=8, shift_size=0, mlp_ratio=4.0
            (norm1): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=180, window_size=(8, 8), num_heads=6
              (qkv): Linear(in_features=180, out_features=540, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=180, out_features=180, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
     

In [ ]:
swin.fit(rekis_train, rekis_dev, args.epochs)

1/60:   0%|                                                  | 0/959 [00:00<?, ?it/s]/home/kostape4/downscale/venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
                                                                                     

Epoch 1/60 - train_loss: 0.6433, dev_loss: 0.2780 (287.35s)


Epoch 2/60 - train_loss: 0.2490, dev_loss: 0.3682 (283.79s)


Epoch 3/60 - train_loss: 0.2116, dev_loss: 0.3340 (283.93s)


Epoch 4/60 - train_loss: 0.1877, dev_loss: 0.3196 (284.13s)


...

Epoch 44/60 - train_loss: 0.0542, dev_loss: 0.0331 (284.42s)


Epoch 45/60 - train_loss: 0.0605, dev_loss: 0.0341 (284.44s)


Epoch 46/60 - train_loss: 0.0510, dev_loss: 0.0317 (284.59s)


Epoch 47/60 - train_loss: 0.0495, dev_loss: 0.0331 (284.45s)


48/60:   3%|█▏                                      | 28/959 [00:07<04:16,  3.64it/s]

In [ ]:
swin.load_weights(os.path.join(args.logdir, swin.model_name))
swin.evaluate(rekis_dev, print_loss=True)

### CORDEX

In [34]:
### FIX
cordex = CORDEX(
    data_path=CORDEX_DATA_PATH,
    variables=args.variables,
    dev_len=("2000-01-01", "2001-01-01"),
    test_len=("2001-01-01", "2002-01-01"),
)

Loading 34 NetCDF file(s)...
Loaded data shape: {'time': 366, 'y': 40, 'x': 40}
Time range: 2000-01-01 12:00:00 to 2000-12-31 12:00:00
Loading 34 NetCDF file(s)...
Loaded data shape: {'time': 365, 'y': 40, 'x': 40}
Time range: 2001-01-01 12:00:00 to 2001-12-31 12:00:00

CORDEX dataset initalized.
dev size: (366)
test size: (365)

